# Tabular Data Science Course Project: Multiple Theoretical Distributions Fitting to Empirical Distribution Using KDE and Modified KS-Tests
## By David Dorfman and Ella Kharakh



### Importing the required libraries:

In [68]:
from scipy.stats import norm, weibull_min, expon
from scipy.optimize import minimize_scalar
from scipy import stats
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import statistics
import math
import warnings
warnings.filterwarnings("ignore")

### Loading the datasets:

In [69]:
patient_survival = pd.read_csv("./dataset1.csv", index_col=0)        # pd.read_csv("./dataset.csv", index_col=0)
climate_data = pd.read_csv("./dataset2.csv", index_col=0)        # pd.read_csv("./daily_data.csv")
predictive_maintenance = pd.read_csv("./dataset3.csv", index_col=0)        # pd.read_csv("./predictive-maintenance-dataset.csv", index_col=0, dtype={'MOU': str})
FIFA_23 = pd.read_csv("./dataset4.csv", index_col=0)        # pd.read_csv("./fifa_23_280922.csv", index_col=0)
datasets = [patient_survival, climate_data, predictive_maintenance, FIFA_23]

### Functions for calculating and plotting the KDE:

In [70]:
def calc_kde(feature):
    kde = sm.nonparametric.KDEUnivariate(feature)
    kde.fit()
    return kde


def show_kde(feature):
    kde = calc_kde(feature)
    samples = np.linspace(min(feature), max(feature), 500)
    log_prob = kde.evaluate(samples)
    plt.plot(samples, np.exp(log_prob), c='green')
    # plt.axis([min(feature), max(feature), -0.1, 1.1])
    # plt.title(feature)
    plt.show()

### Finding the minimum and maximum points for each feature's KDE:

In [71]:
def min_max(data):
    kde = calc_kde(data)
    samples = np.linspace(min(data), max(data), 500)
    vals = np.exp(kde.evaluate(samples))
    min_list, max_list = [], []
    theLastMaxIndex = -1
    theLastMinIndex = -1
    for i in range(3, len(vals) - 3, 1):
        if vals[i] == np.max(vals[i - 3: i + 3]):
            max_list.append((samples[i], vals[i]))
            theLastMaxIndex = i
        elif vals[i] == np.min(vals[i - 3: i + 3]):
            min_list.append((samples[i], vals[i]))
            theLastMinIndex = i
        if (len(min_list) == 0) and (len(max_list) != 0):
            min_list.append((samples[0], vals[0]))
        if (len(min_list) != 0) and (len(max_list) == 0):
            max_list.append((samples[0], vals[0]))
    if theLastMaxIndex > theLastMinIndex:
        min_list.append((samples[len(samples)-1], vals[len(samples)-1]))
    else:
        max_list.append((samples[len(samples)-1], vals[len(samples)-1]))
    return min_list, max_list

### Filtering the Datasets:

In [72]:
def filter_dataset(dataset):
    num_features = dataset.select_dtypes(include=[np.number])
    column_names = list(num_features.columns.values)
    uni_features = [f for f in column_names if dataset[f].nunique() > 50 and dataset[f].isna().sum() / len(dataset[f])\
                    < 0.4 and dataset[f].nunique() < len(dataset[f])]
    min_max_features = {}
    for f in uni_features:
        feature = dataset[f].fillna(dataset[f].mean())
        min_max_features[f] = min_max(feature.to_numpy())
    rel_features = [f for f in min_max_features if len(min_max_features[f][0]) > 1 and len(min_max_features[f][1]) > 1]
    return rel_features


filtered_patient = filter_dataset(patient_survival)
filtered_climate = filter_dataset(climate_data)
filtered_maintenance = filter_dataset(predictive_maintenance)
filtered_FIFA = filter_dataset(FIFA_23)

### Ploting the relevant KDEs for the datasets:

In [73]:
print("*** patient survival dataset ***")

for feature in filtered_patient:
    print(feature)
    show_kde(patient_survival[feature])

print("*** climate data dataset ***")

for feature in filtered_climate:
    print(feature)
    show_kde(climate_data[feature])

print("*** predictive maintenance dataset ***")

for feature in filtered_maintenance:
    print(feature)
    show_kde(predictive_maintenance[feature])

print("*** FIFA 23 dataset ***")

for feature in filtered_FIFA:
    print(feature)
    show_kde(FIFA_23[feature])

*** patient survival dataset ***
*** climate data dataset ***
*** predictive maintenance dataset ***
*** FIFA 23 dataset ***


### Functions for the KS tests:

In [53]:
def ks_test_exp(empirical):
    params = stats.expon.fit(empirical)
    result = stats.kstest(empirical, lambda x: stats.expon.cdf(x, loc=params[0], scale=params[1]))
    return result


def ks_test_weibull(empirical):
    params = stats.weibull_min.fit(empirical)
    return stats.kstest(empirical, lambda x: stats.weibull_min.cdf(x, c=params[0], loc=params[1], scale=params[2]))


def ks_test_normal(empirical):
    params = stats.norm.fit(empirical)
    return stats.kstest(empirical, lambda x: stats.norm.cdf(x, loc=params[0], scale=params[1]))

### Function for getting a subset of a feature:

In [54]:
def subSetOfDomain(ourFeature, intervalSmallerParm, intervalBigParm):
    sub_list = ourFeature
    listToReturn = list()
    for val in sub_list:
        if (val <= intervalBigParm and val >= intervalSmallerParm):
            listToReturn.append(val)
    return listToReturn

### Function for calculating the weighted average of the P-values:

In [55]:
def weightedArithmeticMeanPvalues(listOfPvalues, listOfTheirWeights):
    sum = 0
    for i in range(len(listOfPvalues)):
        sum += listOfPvalues[i] * listOfTheirWeights[i]
    return sum

### Function for evaluating the best P-value:

In [56]:
def pValueEvaluator(feature, begin, end):
    ourInterval = subSetOfDomain(feature, begin, end)
    if len(ourInterval) / len(feature) > 0.05:
        result1 = ks_test_exp(ourInterval).pvalue
        result2 = ks_test_weibull(ourInterval).pvalue
        result3 = ks_test_normal(ourInterval).pvalue
        tempMax = max(result1, result2, result3)
        if tempMax < 0.05:
            return 0, "no theoretical distribution"
        if result1 == tempMax:
            return result1, "Exponential distribution"
        if result2 == tempMax:
            return result2, "Weibull distribution"
        if result3 == tempMax:
            return result3, "Normal distribution" 
    return 0, " "

### Function for checking if we should expand or narrow the interval by 10% or keep the interval as is:

In [57]:
def isBetterRangOfPvalue(ourFeature, begin, end):
    baseFeature = (begin, end)
    num1 = updatenumberToCeil(end,  1.1)
    if num1 >= len(ourFeature):
        num1 = len(ourFeature)-1
    num2 = updatenumberToFloor(begin, 0.9)
    if num2 < 0:
        num2 = 0
    num3 = updatenumberToFloor(end, 0.9)
    num4 = updatenumberToCeil(begin, 1.1)
    valueBaseFeature = pValueEvaluator(ourFeature, begin, end)
    valf1 = pValueEvaluator(ourFeature, begin, num1)
    valf2 = pValueEvaluator(ourFeature, num2, end)
    valf3 = pValueEvaluator(ourFeature, begin, num3)
    valf4 = pValueEvaluator(ourFeature, num4, end)
    maxValue = max(valueBaseFeature[0], valf1[0], valf2[0], valf3[0], valf4[0])
    if maxValue == valueBaseFeature[0]:
        return baseFeature, valueBaseFeature
    if maxValue == valf1[0]:
        return (begin, num1), valf1
    if maxValue == valf2[0]:
        return (num2, end), valf2
    if maxValue == valf3[0]:
        return (begin, num3), valf3
    return (num4, end), valf4

### Helper functions for rounding:

In [58]:
def updatenumberToCeil(num, precentageofchange):
    num = num * precentageofchange
    return math.ceil(num)


def updatenumberToFloor(num, precentageofchange):
    num = num * precentageofchange
    return math.floor(num)

### Function for finding the best ranges based on the P-values:

In [59]:
def bestPvalueRanges(feature):
    min_list, max_list = min_max(feature)
    listOfRanges = list()
    j = 0
    i = 0
    while i < len(min_list) and j < len(max_list):
        if min_list[i][0] < max_list[j][0]:
            listOfRanges.append(((min_list[i][0], max_list[j][0]), pValueEvaluator(feature, min_list[i][0], max_list[j][0])))
            i = i + 1
            if i < len(min_list):
                listOfRanges.append(((min_list[i-1][0], min_list[i][0]), pValueEvaluator(feature, min_list[i-1][0], min_list[i][0])))
        else:
            listOfRanges.append(((max_list[j][0], min_list[i][0]), pValueEvaluator(feature, max_list[j][0], min_list[i][0])))
            j = j + 1
    return changeIntervals(listOfRanges, feature)

### Function for plotting the KDE divided by distributions:

In [60]:
def showGraphOfDistrebutions(intervals, feature, name):
    kde = calc_kde(feature)
    samples = np.linspace(0, max(feature), 6000)
    log_prob = kde.evaluate(samples)
    x = np.sort(feature)
    y = np.exp(log_prob)
    colors = ['blue', 'green', 'orange', 'pink', 'red', 'purple', 'pink', 'turquoise']
    figure, axes = plt.subplots()
    for i, interval in enumerate(intervals):
        j = i
        if i > len(colors) - 1:
            j = i % len(colors)
        mask = np.logical_and(samples >= interval[0][0], samples <= interval[0][1])
        plt.plot(samples[mask], np.exp(log_prob[mask]), c=colors[i])
    axes.legend()
    axes.set_title(str(name))
    for i, interval in enumerate(intervals):
        x_center = np.mean(interval[0])
        y_center = np.max(y) * 1.2
        axes.text(x_center, y_center, f'{interval[1][1]}', ha='center')
    plt.show()

### Function for picking the best intervals to start from:

In [61]:
def bestIntervalsToCheck(listOfRanges, feature):
    listToReturn = list()
    lengthOfFeature = max(feature) - min(feature)
    isFirstTime = 1
    for i in range(len(listOfRanges)):
        if i+1 < len(listOfRanges) and i+2 < len(listOfRanges):
            if listOfRanges[i][0][0] == listOfRanges[i+1][0][0] and listOfRanges[i+2][0][1] == listOfRanges[i+1][0][1]:
                lengthOfInterval1 = (listOfRanges[i][0][1] - listOfRanges[i][0][0])/lengthOfFeature
                lengthOfInterval2 = (listOfRanges[i+1][0][1] - listOfRanges[i+1][0][0])/lengthOfFeature
                lengthOfInterval3 = (listOfRanges[i+2][0][1] - listOfRanges[i+2][0][0])/lengthOfFeature
                weightedPvalue1 = lengthOfInterval1 * listOfRanges[i][1][0]
                weightedPvalue2 = lengthOfInterval2 * listOfRanges[i+1][1][0]
                weightedPvalue3 = lengthOfInterval3 * listOfRanges[i+2][1][0]
                tempMax = max(weightedPvalue1, weightedPvalue2, weightedPvalue3)
                if weightedPvalue1 == tempMax:
                    listToReturn.append(listOfRanges[i])
                elif weightedPvalue2 == tempMax:
                    listToReturn.append(listOfRanges[i+1])
                elif weightedPvalue3 == tempMax:
                    listToReturn.append(listOfRanges[i+2])
                isFirstTime = 0
                if i+2 == len(listOfRanges) - 1:
                    i = len(listOfRanges)
            elif isFirstTime or i == len(listOfRanges) - 1:
                isFirstTime = 0
                listToReturn.append(listOfRanges[i])
    return listToReturn


def bestVersionOfInterval(rang, feature):
    pValueinteval1 = rang
    tempPvalue = isBetterRangOfPvalue(feature, rang[0][0], rang[0][1])
    while (tempPvalue[1][0] > pValueinteval1[1][0]):
        pValueinteval1 = tempPvalue
        tempPvalue = isBetterRangOfPvalue(feature, pValueinteval1[0][0], pValueinteval1[0][1])
    return pValueinteval1


def adjustIntervals(listOfRanges, feature):
    listToReturn = list()
    for rang in listOfRanges:
        rang11 = bestVersionOfInterval(rang, feature)
        if rang11[1][0] != 0:
            listToReturn.append(rang11)
    return listToReturn

### Functions for treatment in overlaps:

In [62]:
def recursiveCareOfOvelap(feature, rang1, rang2, pvalue1, reqDepth):
    num1 = ((rang1[0][1] - rang2[0][0])/2)
    pValueInterval1 = (rang2[0][0] + num1 - rang1[0][0]) * (pValueEvaluator(feature, rang1[0][0], rang2[0][0] + num1)[0])
    pValueInterval2 = (rang2[0][1] - rang2[0][0] - num1) * (pValueEvaluator(feature, rang2[0][0] + num1, rang2[0][1])[0])
    pvalueNew = pValueInterval1 + pValueInterval2
    if pvalue1 - pvalueNew >= 0.01 or reqDepth >= 300:
        return rang1, rang2
    else:
        rang1 = ((rang1[0][0], rang2[0][0] + num1), pValueEvaluator(feature, rang1[0][0], rang2[0][0] + num1))
        rang2 = ((rang2[0][0] + num1, rang2[0][1]), pValueEvaluator(feature, rang2[0][0] + num1, rang2[0][1]))
        pvalue1 = pvalueNew
        return recursiveCareOfOvelap(feature, rang1, rang2, pvalue1, reqDepth + 1)
    

def careOfOverlap(interval1, interval2, feature):
    num1 = ((interval1[0][1] - interval2[0][0])/2)
    pValueInterval1 = (interval2[0][0] + num1 - interval1[0][0]) * (pValueEvaluator(feature, interval1[0][0], interval2[0][0] + num1)[0])
    pValueInterval2 = (interval2[0][1] - interval2[0][0] - num1) * (pValueEvaluator(feature, interval2[0][0] + num1, interval2[0][1])[0])
    pvalueNew = pValueInterval1 + pValueInterval2
    pvalueOriginal = ((interval1[0][1] - interval1[0][0]) * interval1[1][0]) + ((interval2[0][1] - interval1[0][1]) * (pValueEvaluator(feature, interval1[0][1], interval2[0][1])[0]))
    if pvalueNew <= pvalueOriginal:
        rang1 = interval1
        rang2 = ((interval1[0][1], interval2[0][1]), pValueEvaluator(feature, interval1[0][1], interval2[0][1]))
        return rang1, rang2
    else:
        rang1 = ((interval1[0][0], interval2[0][0] + num1), pValueEvaluator(feature, interval1[0][0], interval2[0][0] + num1))
        rang2 = ((interval2[0][0] + num1, interval2[0][1]), pValueEvaluator(feature, interval2[0][0] + num1, interval2[0][1]))
        return recursiveCareOfOvelap(feature, rang1, rang2, pvalueNew, 0)


def eraseOverlaps(listOfRanges, feature):
    i = 1
    newList = list()
    currentRange = listOfRanges[0]
    while i < len(listOfRanges):
        nextRange = listOfRanges[i]
        if ((currentRange[0][0] <= nextRange[0][0]) and (currentRange[0][1] >= nextRange[0][0])) or ((currentRange[0][0] >= nextRange[0][1]) and (currentRange[0][1] >= nextRange[0][1])):
            rang1, rang2 = careOfOverlap(currentRange, nextRange, feature)
            newList.append(rang1)
            currentRange = rang2
        else:
            newList.append(currentRange)
            currentRange = nextRange
        i = i + 1
    if currentRange not in newList:
        newList.append(currentRange)
    return newList

### Functions for treatment in gaps:

In [63]:
def recursiveCareOfGaps(feature, interval1, interval2, pvalue1, reqDepth):
    num1 = ((interval1[0][1] - interval2[0][0])/2)
    pValueInterval1 = (interval1[0][1] + num1 - interval1[0][0]) * (pValueEvaluator(feature, interval1[0][0], interval1[0][1] + num1)[0])
    pValueInterval2 = (interval2[0][1] - interval2[0][0] + num1) * (pValueEvaluator(feature, interval2[0][0] - num1, interval2[0][1])[0])
    pvalueNew = pValueInterval1 + pValueInterval2
    if pvalue1 - pvalueNew >= 0.000001 or reqDepth>=300:
        return interval1, interval2
    else:
        rang1 = ((interval1[0][0], interval1[0][1] + num1), pValueEvaluator(feature, interval1[0][0], interval1[0][1] + num1))
        rang2 = ((interval2[0][0] - num1, interval2[0][1]), pValueEvaluator(feature, interval2[0][0] - num1, interval2[0][1]))
        pvalue1 = pvalueNew
        return recursiveCareOfGaps(feature, rang1, rang2, pvalueNew, reqDepth + 1)


def careOfGaps(interval1, interval2, feature):
    num1 = ((interval2[0][0] - interval1[0][1])/2)
    pValueInterval1 = (interval1[0][1] + num1 - interval1[0][0]) * (pValueEvaluator(feature, interval1[0][0], interval1[0][1] + num1)[0])
    pValueInterval2 = (interval2[0][1] - interval2[0][0] + num1) * (pValueEvaluator(feature, interval2[0][0] - num1, interval2[0][1])[0])
    pvalueNew = pValueInterval1 + pValueInterval2
    pvalueOriginal1 = ((interval1[0][1] - interval1[0][0]) * interval1[1][0]) + ((interval2[0][1] - interval1[0][1]) * (pValueEvaluator(feature, interval1[0][1], interval2[0][1])[0]))
    pvalueOriginal2 = ((interval2[0][1] - interval2[0][0]) * interval2[1][0]) + ((interval2[0][0] - interval1[0][0]) * (pValueEvaluator(feature, interval1[0][0], interval2[0][0])[0]))
    tempMax = max(pvalueOriginal1, pvalueOriginal2, pvalueNew)
    if pvalueOriginal1 == tempMax:
        rang1 = interval1
        rang2 = ((interval1[0][1], interval2[0][1]), pValueEvaluator(feature, interval1[0][1], interval2[0][1]))
        return rang1, rang2
    elif pvalueOriginal2 == tempMax:
        rang2 = interval2
        rang1 = (interval1[0][0], interval2[0][0]), pValueEvaluator(feature, interval1[0][0], interval2[0][0])
        return rang1, rang2
    else:
        rang1 = ((interval1[0][0], interval1[0][1] + num1), pValueEvaluator(feature, interval1[0][0], interval1[0][1] + num1))
        rang2 = ((interval2[0][0] - num1, interval2[0][1]), pValueEvaluator(feature, interval2[0][0] - num1, interval2[0][1]))
        return recursiveCareOfGaps(feature, rang1, rang2, pvalueNew, 0)


def eraseGaps(listOfRanges, feature):
    i = 1
    newList = list()
    currentRange = listOfRanges[0]
    if currentRange[0][0] != 0:
        currentRange = ((0, currentRange[0][1]), pValueEvaluator(feature, 0, currentRange[0][1]))
    while i < len(listOfRanges):
        nextRange = listOfRanges[i]
        if currentRange[0][1] < nextRange[0][1]:
            rang1, rang2 = careOfGaps(currentRange, nextRange, feature)
            newList.append(rang1)
            currentRange = rang2
        else:
            newList.append(currentRange)
            currentRange = nextRange
        i = i + 1
    if currentRange not in newList:
        newList.append(currentRange)
    return newList

### Function for finding the best partition to distributions of the KDE:

In [64]:
def changeIntervals(listOfRanges, feature):
    listOfRangesUpdated = bestIntervalsToCheck(listOfRanges, feature)
    listOfRangesUpdated = adjustIntervals(listOfRangesUpdated, feature)
    listOfRangesUpdated = eraseOverlaps(listOfRangesUpdated, feature)
    listOfRangesUpdated = eraseGaps(listOfRangesUpdated, feature)
    return listOfRangesUpdated

### Function for calculating the score of our method:

In [65]:
def upgradedKsCalc(feature):
    ranges = bestPvalueRanges(feature)
    listOfWeights = list()
    listOfPvalues = list()
    lengthOfFeature = max(feature)
    for i in range(len(ranges)):
        listOfPvalues.append(ranges[i][1][0])
        listOfWeights.append((ranges[i][0][1] - ranges[i][0][0])/lengthOfFeature)
    pvalueOfFeature = weightedArithmeticMeanPvalues(listOfPvalues, listOfWeights)
    return pvalueOfFeature

### Comparing the results between our solution and the regular solution:

In [67]:
def compare(feature):
    ourSolution = upgradedKsCalc(feature)
    traditionalSolution = max(ks_test_exp(feature).pvalue, ks_test_normal(feature).pvalue, ks_test_weibull(feature).pvalue)
    print("our solution: " + str(ourSolution))
    print("original solution: " + str(traditionalSolution))
    ranges = bestPvalueRanges(feature)
    showGraphOfDistrebutions(ranges, feature, "sample")


print("*** patient survival dataset ***")
for feature in filtered_patient:
    compare(patient_survival[feature])

print("*** climate data dataset ***")
for feature in filtered_climate:
    compare(climate_data[feature])

print("*** predictive maintenance dataset ***")
for feature in filtered_maintenance:
    compare(predictive_maintenance[feature])

print("*** FIFA 23 dataset ***")
for feature in filtered_FIFA:
    compare(FIFA_23[feature])

*** patient survival dataset ***


IndexError: list index out of range